In [1]:
import base58
import binascii
import ecdsa
import hashlib
import json
import os
import subprocess
import time

def dSHA256(data):
    hash_1 = hashlib.sha256(data).digest()
    hash_2 = hashlib.sha256(hash_1).digest()
    return hash_2

def hash160(s):
    '''sha256 followed by ripemd160'''
    return hashlib.new('ripemd160', hashlib.sha256(s).digest()).digest()

def privkey_to_pubkey(privkey):
    signing_key = ecdsa.SigningKey.from_string(privkey, curve=ecdsa.SECP256k1) # Don't forget to specify the curve
    verifying_key = signing_key.get_verifying_key()

    # Use this code block if the address you gave corresponds to the compressed public key
    x_cor = bytes.fromhex(verifying_key.to_string().hex())[:32] # The first 32 bytes are the x coordinate
    y_cor = bytes.fromhex(verifying_key.to_string().hex())[32:] # The last 32 bytes are the y coordinate
    if int.from_bytes(y_cor, byteorder="big", signed=True) % 2 == 0: # We need to turn the y_cor into a number.
        public_key = bytes.fromhex("02" + x_cor.hex())
    else:
        public_key = bytes.fromhex("03" + x_cor.hex())
    return public_key

In [2]:
# Functions related to generating bitcoin addresses

def encode_base58(s):
    BASE58_ALPHABET = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
    count = 0
    for c in s:  
        if c == 0:
            count += 1
        else:
            break
    num = int.from_bytes(s, 'big')
    prefix = '1' * count
    result = ''
    while num > 0:  
        num, mod = divmod(num, 58)
        result = BASE58_ALPHABET[mod] + result
    return prefix + result

def encode_base58_checksum(b):
    return encode_base58(b + dSHA256(b)[:4])

# https://en.bitcoin.it/wiki/Wallet_import_format
def privkey_to_wif(privkey, compressed_pubkey, testnet):
    if testnet:
        prefix = b"\xEF"
    else:
        prefix = b"\x80"
    
    # if the privkey will correspond to a compressed public key
    if compressed_pubkey: 
        extended = prefix + privkey + b"\x01"
    else:
        extended = prefix + privkey 
        
    extendedchecksum = extended + dSHA256(extended)[:4]
    wif = encode_base58(extendedchecksum)
    
    return wif

# https://learnmeabitcoin.com/guide/wif
def wif_to_privkey(private_key_WIF):
    private_key_full = base58.b58decode(private_key_WIF)
    
    # If the WIF encoding includes the optional "01" byte for compressed privKey,
    # do not include it in the final output.
    if len(private_key_full) == 38:
        private_key = private_key_full[1:-5] 
        print("compressed pubkey")
    else:
        private_key = private_key_full[1:-4] 
        print("not compressed pubkey")
    return private_key


def pk_to_p2pkh(compressed, testnet):
    '''Returns the address string'''
    pk_hash = hash160(compressed)
    if testnet:
        prefix = b'\x6f'
    else:
        prefix = b'\x00'
    return encode_base58_checksum(prefix + pk_hash)

def pk_to_p2sh_p2wpkh(compressed, testnet):
    pk_hash = hash160(compressed)
    redeemScript = bytes.fromhex(f"0014{pk_hash.hex()}")
    rs_hash = hash160(redeemScript)
    if testnet:
        prefix = b"\xc4"
    else:
        prefix = b"\x05"
    return encode_base58_checksum(prefix + rs_hash)

def wif_to_addresses(wif):
#     wif = "cUy9rC6wteKizfu1fgP2abKUWTkJxjqKp2fba91FkU332CFHo6ix"
    privkey = wif_to_privkey(wif)
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, testnet = True)
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

    print("WIF Private key: " + wif)
    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)
    
def privkey_to_addresses(privkey):
    # privkey = bytes.fromhex("AF933A6C602069F1CBC85990DF087714D7E86DF0D0E48398B7D8953E1F03534A")
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, testnet = True)
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)

# Merch close v2 - separate input/output for fees

Generate all the priv/public keys

In [3]:
funding_privkey_hex = "1111111111111111111111111111111100000000000000000000000000000000"
funding_pubkey_hex = privkey_to_pubkey(bytes.fromhex(funding_privkey_hex)).hex()

change_privkey_hex = "1111111111111111111111111111111111111111111111111111111111111111"
change_pubkey_hex = privkey_to_pubkey(bytes.fromhex(change_privkey_hex)).hex()

merch_privkey_hex = "3911111111111111111111111111111111111111111111111111111111111111"
merch_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_privkey_hex)).hex()

cust_privkey_hex = "7911111111111111111111111111111111111111111111111111111111111111"
cust_pubkey_hex = privkey_to_pubkey(bytes.fromhex(cust_privkey_hex)).hex()

# cust_payout_privkey_hex = "7711111111111111111111111111111111111111111111111111111111111111"
# cust_payout_pubkey_hex = privkey_to_pubkey(bytes.fromhex(cust_payout_privkey_hex)).hex()

merch_payout_privkey_hex = "3711111111111111111111111111111111111111111111111111111111111111"
merch_payout_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_payout_privkey_hex)).hex()

# merch_disp_privkey_hex = "3111111111111111111111111111111111111111111111111111111111111111"
# merch_disp_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_disp_privkey_hex)).hex()

# revocation_secret_hex = "4011111111111111111111111111111111111111111111111111111111111111"
# RL = hashlib.sha256(bytes.fromhex(revocation_secret_hex)).digest()
# revocation_lock_hex = RL.hex()

# merch_cpfp_privkey_hex = "2222222222222222222222222222222277777777777777777777777777777777"
# merch_cpfp_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_cpfp_privkey_hex)).hex()

# merch_fee_privkey_hex = "2222222222222222222222222222222266666666666666666666666666666666"
# merch_fee_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_fee_privkey_hex)).hex()

### Start up regtest mode
Delete any history so we are starting from scratch. <br>
Mine 101 blocks

In [4]:
# Make sure bitcoind is not already running
os.system("bitcoin-cli -regtest stop")
time.sleep(2) 

# Delete any previous files to restart regtest
os.system("rm -rfv $HOME/Library/Application\ Support/Bitcoin/regtest/")

# start up bitcoind in regtest mode
os.system("bitcoind -regtest -daemon -minrelaytxfee=0")
time.sleep(2)

# generate 101 blocks so we can fund transactions
os.system("bitcoin-cli -regtest generate 101")
blockcount = subprocess.getoutput("bitcoin-cli -regtest getblockcount")

print("blockcount: " + str(blockcount))

blockcount: 101


Generate base58 address for the escrow funder

In [5]:
# Generate p2sh-p2wpkh address to fund the escrow funder
privkey = bytes.fromhex(funding_privkey_hex)
public_key = privkey_to_pubkey(privkey)
p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

# print("Private key: " + privkey.hex())
# print("Public key: " + public_key.hex())
print("Address: " + p2sh_p2wpkh_address)

Address: 2MuuVDMJqfh2J9iS3DtfUeU1tEfff1SNahi


Send btc to the escrow funder

In [6]:
txid_1 = subprocess.getoutput("bitcoin-cli -regtest sendtoaddress " + p2sh_p2wpkh_address + " 3.0")
print(txid_1)

dd98f509f354ca3a6a64b5c6bcb22c9a49482dabac2f9442cb07426e6daf7e5e


In [7]:
# Find which output index the btc was sent to
raw_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + txid_1)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_tx)
d = json.loads(decoded)
# print(decoded)

if d["vout"][0]["scriptPubKey"]["addresses"][0] == p2sh_p2wpkh_address:
    index = 0
else:
    index = 1
print("index: " + str(index))

index: 1


In [8]:
os.system("bitcoin-cli -regtest generate 1");

## Funding tx

In [9]:
raw_escrow_tx = subprocess.getoutput("python funding_tx_with_changev2.py" 
                        + " --txid " + txid_1
                        + " --index " + str(index)
                        + " --input_amount_btc " + "3.0"
                        + " --funding_privkey " + funding_privkey_hex
                        + " --escrow_value_btc " + "2"
                        + " --cust_pubkey " + cust_pubkey_hex
                        + " --merch_pubkey " + merch_pubkey_hex
                        + " --cust_change_value_btc " + "1"
                        + " --cust_change_pubkey " + change_pubkey_hex)
print("serialized funding tx:\n" + raw_escrow_tx)

escrow_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_escrow_tx + " true")
# "true" flag means we are okay with absurdly high tx fee

print("\nfunding txid:\n"+escrow_txid)

serialized funding tx:
020000000001015e7eaf6d6e4207cb42942facab2d48499a2cb2bcc6b5646a3aca54f309f598dd0100000017160014578dd1183845e18d42f90b1a9f3a464675ad2440ffffffff0200c2eb0b00000000220020666c6bfa88ba97c90cb04c7038d56b5854e71a4dd174d79b1260c822a14f791e00e1f50500000000160014fc7250a211deddc70ee5a2738de5f07817351cef0247304402207128284bdfb3fd1e01cdea6a86247ee2970e57160ff9b95c2a5259e1f7a6494702206837808ba5be986e41ecbe85e2a644b6c017ea6f8f2e1c944440f6d316f068390121021d5c5194a62b272f98a7d8321e4b3a03add9d61d13a15a00123888b81850cee200000000

funding txid:
a1d58b6f0e1383b52726807c783d1ad12b0638ead1a6184368eca30fd0352a22


Broadcast escrow funding transaction

In [10]:
# raw_escrow_tx1 = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + escrow_txid)
# decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_escrow_tx1)
# print(decoded)

In [11]:
os.system("bitcoin-cli -regtest generate 1");

## Cust and Merch sign merch-close with sighash Single|AnyoneCanPay

In [12]:
# SIGHASH SINGLE & ANYONE CAN PAY
merch_close_acp_tx = subprocess.getoutput("python merch_close_anyonecanpay.py"        
                + " --txid_str " + escrow_txid
                + " --index " + "0"
                + " --input_amount_btc " + "2"
                + " --cust_privkey " + cust_privkey_hex 
                + " --merch_privkey " + merch_privkey_hex
                + " --sighash_type " + "82"
                + " --output_value_btc " + "2"
                + " --merch_payout_pubkey " + merch_payout_pubkey_hex   
                + " --to_self_delay " + "05cf") # number of blocks to delay to-self output       
print("merch close anyonecanpay tx:\n" + merch_close_acp_tx)

merch close anyonecanpay tx:
02000000000101222a35d00fa3ec684318a6d1ea38062bd11a3d787c802627b583130e6f8bd5a10000000000ffffffff0100c2eb0b00000000220020c3fae9ae705465ac132b128c84fc011be28c21bff28e165f7cfb776dfbb117ff0400483045022100bf9a5cd957ca3285c9a4b434817c70f813eb403cff6fdb63c1d4dbb09787f04c02200848962c55b952edb5b6dd54ff95825764607151a7d4090cbdda2fba3dc4249e8248304502210099db84f4eba58cbebf8abdb10e5e9d119bfcd611df4f52bd9393f7abd70ab69c02203857eba596201a85738f0122cd07565988dd95f140bd19f34fd796ae706d4faa8247522102f3d17ca1ac6dcf42b0297a71abb87f79dfa2c66278cbb99c1437e6570643ce902103fc43b44cd953c7b92726ebefe482a272538c7e40fdcde5994a62841525afa8d752ae00000000


If we wanted to, we could broadcast this tx by itself and it would be a valid tx (cell below). Instead, we will combine it with the merch's tx.

In [13]:
# merch_close_acp_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + merch_close_acp_tx)
# print(merch_close_acp_txid) 

In [14]:
# os.system("bitcoin-cli -regtest generate 1");

In [15]:
# mined_merch_close_acp_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + merch_close_acp_txid)
# decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + mined_merch_close_acp_tx)
# print(decoded)

## CREATE MERCH FEE P2WPKH INPUT

Below are three privkey/pubkeys the merchant will use. <br> 
1 - address to fund the merch <br>
2 - input for the fee tx <br>
3 - output for the fee tx

In [16]:
merch1_privkey_hex = "0221111111111111111111111111111100000000000000000000000000000000"
merch1_privkey = bytes.fromhex(merch1_privkey_hex)
merch1_pubkey = privkey_to_pubkey(merch1_privkey)

merch2_privkey_hex = "2221111111111111111111111111111100000000000000000000000000000000"
merch2_privkey = bytes.fromhex(merch2_privkey_hex)
merch2_pubkey = privkey_to_pubkey(merch2_privkey).hex()

merch3_privkey_hex = "3311111111111111111111111111111100000000000000000000000000000000"
merch3_privkey = bytes.fromhex(merch3_privkey_hex)
merch3_pubkey = privkey_to_pubkey(merch3_privkey).hex()

# address to fund the merchant
merch1_p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(merch1_pubkey, testnet = True)
print("Address: " + merch1_p2sh_p2wpkh_address)

Address: 2NBEoMBERqETXQ3jiHrVGbZvQgKGx4PQppr


In [17]:
# Fund the merchant
txid_2 = subprocess.getoutput("bitcoin-cli -regtest sendtoaddress " + merch1_p2sh_p2wpkh_address + " 1")
print("txid:" + txid_2)

# Find which output index the btc was sent to
raw_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + txid_2)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_tx)
d = json.loads(decoded)
# print(decoded)

if d["vout"][0]["scriptPubKey"]["addresses"][0] == merch1_p2sh_p2wpkh_address:
    index = 0
else:
    index = 1
print("index: " + str(index))

txid:11f89c94b95c0d6e25dc51a0ad248f5a05f6a845a9c0a732b1ed3b5b66568391
index: 1


In [18]:
os.system("bitcoin-cli -regtest generate 1");

Fund the input to merch fee tx

In [19]:
merch1_tx = subprocess.getoutput("python p2nsh_to_p2wpkh.py" 
                        + " --txid " + txid_2
                        + " --index " + str(index)
                        + " --input_amount_btc " + "1"
                        + " --input_privkey " + merch1_privkey_hex
                        + " --payout_value_btc " + "1"
                        + " --payout_pubkey " + merch2_pubkey)
# print(merch1_tx)

merch1_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + merch1_tx + " true")
# "true" flag means we are okay with absurdly high tx fee
print(merch1_txid)

109e272d4ccd42f90d3856714afb05948428cbee19e7733f317f6bc96f3963a3


In [20]:
# merch_tx1 = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + merch1_txid)
# decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + merch_tx1)
# print(decoded)

## Create Merch Fee tx (p2wpkh to p2wpkh)

Here, we generate a tx with just one input and one output, except the digest also contains the output from the merch_close_anyonecanpay, since we want to make sure the fees can only be added to that tx and not anyone else's.

In [21]:
merch_fee_tx = subprocess.getoutput("python P2WPKH_to_P2WPKH_sighash_all_anyonecanpay.py" 
                        + " --txid_str " + merch1_txid
                        + " --index " + "0"
                        + " --input_amount_btc " + " 1"
                        + " --privkey " + merch2_privkey_hex
                        + " --sighash_type " + "01"
                        + " --output_value_btc " + " 1"
                        + " --payout_pubkey " + merch3_privkey_hex
                        + " --merch_close_tx " + merch_close_acp_tx)
print(merch_fee_tx)

02000000000101a363396fc96b7f313f73e719eecb28849405fb4a7156380df942cd4c2d279e100000000000ffffffff0100e1f505000000001600146a8cbbddc82aafb86a166ab0e2c464c3a5e8766b02483045022100d7b801f626705945db3aff45ecd91c6f87676bcee1938e596a7738eecaeee13d02204ce31ef7a879f7921c54eaa298ccd74e034dd5bc71d3649640af7eaab2d195e3012102c0947a47a59cb42316750ddd23d506d4c23ca997fbe40e9cb813970940501f4f00000000


If the tx was signed with sighash none, or if the merch_close output was not included in the digest, we could broadcast this tx by itself (cell below). 

In [22]:
# merch2_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + merch2_tx + " true")
# print(merch2_txid)

In [23]:
# merch_tx2 = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + merch2_txid)
# decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + merch_tx2)
# print(decoded)

## Combine the two transactions (merch-close and fee)

In [24]:
print("merch_close_acp_tx:\n" + merch_close_acp_tx)
# merch_close_acp_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_merch_close_acp_tx)
# print(merch_close_acp_txid) 

print("\nmerch_fee_tx:\n" + merch_fee_tx)
# merch2_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + merch2_tx)
# print(merch2_txid)

merch_close_acp_tx:
02000000000101222a35d00fa3ec684318a6d1ea38062bd11a3d787c802627b583130e6f8bd5a10000000000ffffffff0100c2eb0b00000000220020c3fae9ae705465ac132b128c84fc011be28c21bff28e165f7cfb776dfbb117ff0400483045022100bf9a5cd957ca3285c9a4b434817c70f813eb403cff6fdb63c1d4dbb09787f04c02200848962c55b952edb5b6dd54ff95825764607151a7d4090cbdda2fba3dc4249e8248304502210099db84f4eba58cbebf8abdb10e5e9d119bfcd611df4f52bd9393f7abd70ab69c02203857eba596201a85738f0122cd07565988dd95f140bd19f34fd796ae706d4faa8247522102f3d17ca1ac6dcf42b0297a71abb87f79dfa2c66278cbb99c1437e6570643ce902103fc43b44cd953c7b92726ebefe482a272538c7e40fdcde5994a62841525afa8d752ae00000000

merch_fee_tx:
02000000000101a363396fc96b7f313f73e719eecb28849405fb4a7156380df942cd4c2d279e100000000000ffffffff0100e1f505000000001600146a8cbbddc82aafb86a166ab0e2c464c3a5e8766b02483045022100d7b801f626705945db3aff45ecd91c6f87676bcee1938e596a7738eecaeee13d02204ce31ef7a879f7921c54eaa298ccd74e034dd5bc71d3649640af7eaab2d195e3012102c0947a47a59cb4231675

In [25]:
merch_close_with_fee_tx = subprocess.getoutput("python combine_merch-close_and_fee.py" 
                        + " " + merch_close_acp_tx
                        + " " + merch_fee_tx
                        )
print("merch_close_with_fee_tx:\n" + merch_close_with_fee_tx)

merch_close_with_fee_tx:
02000000000102222a35d00fa3ec684318a6d1ea38062bd11a3d787c802627b583130e6f8bd5a10000000000ffffffffa363396fc96b7f313f73e719eecb28849405fb4a7156380df942cd4c2d279e100000000000ffffffff0200c2eb0b00000000220020c3fae9ae705465ac132b128c84fc011be28c21bff28e165f7cfb776dfbb117ff00e1f505000000001600146a8cbbddc82aafb86a166ab0e2c464c3a5e8766b0400483045022100bf9a5cd957ca3285c9a4b434817c70f813eb403cff6fdb63c1d4dbb09787f04c02200848962c55b952edb5b6dd54ff95825764607151a7d4090cbdda2fba3dc4249e8248304502210099db84f4eba58cbebf8abdb10e5e9d119bfcd611df4f52bd9393f7abd70ab69c02203857eba596201a85738f0122cd07565988dd95f140bd19f34fd796ae706d4faa8247522102f3d17ca1ac6dcf42b0297a71abb87f79dfa2c66278cbb99c1437e6570643ce902103fc43b44cd953c7b92726ebefe482a272538c7e40fdcde5994a62841525afa8d752ae02483045022100d7b801f626705945db3aff45ecd91c6f87676bcee1938e596a7738eecaeee13d02204ce31ef7a879f7921c54eaa298ccd74e034dd5bc71d3649640af7eaab2d195e3012102c0947a47a59cb42316750ddd23d506d4c23ca997fbe40e9cb813970

In [26]:
merch_close_with_fee_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + merch_close_with_fee_tx + " true")
# "true" flag means we are okay with absurdly high tx fee
print(merch_close_with_fee_txid)

4b6c0bcb19b8a9b0458a720168b1236833df11c2f201a0facc5cb7e88a78b181


In [27]:
os.system("bitcoin-cli -regtest generate 1");

## Mined merch_close_with_fee_tx

In [28]:
close_with_fee_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + merch_close_with_fee_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + close_with_fee_tx)
print(decoded)

{
  "txid": "4b6c0bcb19b8a9b0458a720168b1236833df11c2f201a0facc5cb7e88a78b181",
  "hash": "fa89064159c1b74a082f461a5414b18f0d1076aca09dab36c1db32e717cea872",
  "version": 2,
  "size": 496,
  "vsize": 249,
  "weight": 994,
  "locktime": 0,
  "vin": [
    {
      "txid": "a1d58b6f0e1383b52726807c783d1ad12b0638ead1a6184368eca30fd0352a22",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "",
        "3045022100bf9a5cd957ca3285c9a4b434817c70f813eb403cff6fdb63c1d4dbb09787f04c02200848962c55b952edb5b6dd54ff95825764607151a7d4090cbdda2fba3dc4249e82",
        "304502210099db84f4eba58cbebf8abdb10e5e9d119bfcd611df4f52bd9393f7abd70ab69c02203857eba596201a85738f0122cd07565988dd95f140bd19f34fd796ae706d4faa82",
        "522102f3d17ca1ac6dcf42b0297a71abb87f79dfa2c66278cbb99c1437e6570643ce902103fc43b44cd953c7b92726ebefe482a272538c7e40fdcde5994a62841525afa8d752ae"
      ],
      "sequence": 4294967295
    },
    {
      "txid": "109e272d4ccd